In [ ]:

from collections import defaultdict
from multiprocessing import Pool

class Doc_Ind:
    def _init_(self, data):
        self.data = data

    def extracts(self, line):
        article_id, section_text = line.split(",", 1)
        words = re.findall(r'\w+', section_text.lower())
        return [(word, article_id) for word in words]

    def count_documents(self, word_article_pairs):

        word_count = defaultdict(set)

        for word, article_id in word_article_pairs:

            word_count[word].add(article_id)

        return [(word, len(articles)) for word, articles in word_count.items()]

    def run(self):
        with Pool() as pool:
            word_article_pairs = sum(pool.map(self.extracts, self.data), [])
            word_counts = self.count_documents(word_article_pairs)
        return word_counts

class Indexer:
    def _init_(self, total_documents):
        self.total_documents = total_documents

    def compute(self, args):
        word, document_count, current_article_id, section_text = args
        word = word.strip('"')
        document_count = int(document_count)
        idf = math.log10(self.total_documents / document_count)

        words = re.findall(r'\w+', section_text.lower())
        tf = words.count(word) / len(words)
        tfidf = tf * idf
        return word, (current_article_id, tfidf)

    def combine(self, tfidf_values):
        tfidf_dict = {}
        for word, tfidf in tfidf_values:
            tfidf_dict[word] = tfidf
        return tfidf_dict

    def run(self, word_document_count, current_article):

        current_article_id, section_text = current_article.split(",", 1)

        tfidf_values = [self.compute_tfidf((word, count, current_article_id, section_text)) for word, count in word_document_count]

        return self.combine_tfidf(tfidf_values)

class Query:

    def _init_(self, query, total_documents):

        self.query = query

        self.total_documents = total_documents


    def calculate_relevance(self, tfidf_values):
        query_vector = {word: idf for word, idf in tfidf_values}
        dot_product = sum(query_vector.get(word, 0) * tfidf for word, tfidf in tfidf_values.values())
        return None, dot_product

    def rank_documents(self, relevance_scores):
        return sorted(relevance_scores, key=lambda x: x[1], reverse=True)

    def run(self, tfidf_values):
        relevance_scores = self.calculate_relevance(tfidf_values)
        return self.rank_documents(relevance_scores)

class QueryRee:
    def _init_(self, data, relevant_documents):
        self.data = data
        self.relevant_documents = relevant_documents

    def fetch_(self, line):
        article_id, section_text = line.split(",", 1)
        if article_id in self.relevant_documents:
            return article_id, section_text

    def run(self):
        relevant_documents = filter(None, map(self.fetch_, self.data))
        return relevant_documents

if _name_ == '_main_':

    test_da = [
        "1,\"Introduction: MapReduce is a programming model and an associated implementation for processing and generating large datasets. It was developed at Google to support distributed computing on large clusters of commodity hardware. The model is inspired by the map and reduce functions commonly used in functional programming, although their purpose in the MapReduce framework is not the same as in their original forms.\"",
        "2,\"Map Function: In the MapReduce framework, the map function takes input key/value pairs and processes each pair to generate a set of intermediate key/value pairs. The output of the map function is then shuffled and sorted to group together pairs with the same intermediate key, which are then passed to the reduce function.\"",
        "3,\"Reduce Function: The reduce function receives a key and a set of associated values, and it processes these values to produce a final result. The output of the reduce function is typically written to a file system or another storage system. The MapReduce framework automatically handles many of the details of distributed computing, such as fault tolerance, data distribution, and load balancing.\"",
        "4,\"Usage: MapReduce is commonly used for processing large-scale datasets in distributed environments. It is particularly well-suited for tasks such as data processing, log analysis, web indexing, and machine learning. Many popular distributed computing frameworks, such as Apache Hadoop and Apache Spark, are based on the MapReduce model.\"",
        "5,\"Example: As an example, consider a word count program implemented using MapReduce. The program would read input text and emit key/value pairs where the key is a word and the value is the number 1. The reduce function would then receive these key/value pairs and sum up the values for each output word, providing a total count of each word in the input text.\""
    ]

    # Document indexing
    dor = Doc_Ind(test_data)

    document_count = dor.run()


    indexer = Indexer(len(test_data))

    tfidf_values = []

    for article in test_data:

        tfidf_values.append(indexer.run(document_count, article))

    print("Document Count:", document_count)

    for i, tfidf in enumerate(tfidf_values, start=1):

        print(f"TF-IDF for Document {i}:", tfidf)

    # Query processing
    query = "map function"
    query_processor = QueryProcessor(query, len(test_data))
    relevance_scores = query_processor.run(tfidf_values)
    print("Relevance Scores:", relevance_scores)

    # Query response
    relevant_documents = ['3', '7', '5']

    query_response = QueryResponse(test_data, relevant_documents)

    relevant_documents = query_response.run()

    print("Output:", list(relevant_documents))